In [1]:
import numpy as np
import pandas as pd
import torch
from torch import Tensor
import torch.nn as nn
from typing import Callable, Any, Optional, Tuple, List
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
import random
import os
from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
import copy
import time
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

In [2]:
# InceptionA and InceptionC
# Copied from https://github.com/pytorch/vision/blob/master/torchvision/models/inception.py
# Copied from https://amaarora.github.io/2020/07/24/SeNet.html

class BasicConv1d(nn.Module):

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        **kwargs: Any
    ) -> None:
        super(BasicConv1d, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm1d(out_channels, eps=0.001)

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv(x)
        x = self.bn(x)
        return F.leaky_relu(x, inplace=True)

    
class InceptionA(nn.Module):

    def __init__(
        self,
        in_channels: int,
        pool_features: int,
        conv_block: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(InceptionA, self).__init__()
        if conv_block is None:
            conv_block = BasicConv1d
        self.branch1x1 = conv_block(in_channels, 64, kernel_size=1)

        self.branch5x5_1 = conv_block(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = conv_block(48, 64, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = conv_block(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = conv_block(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = conv_block(96, 96, kernel_size=3, padding=1)

        self.branch_pool = conv_block(in_channels, pool_features, kernel_size=1)

    def _forward(self, x: Tensor) -> List[Tensor]:
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)


class InceptionC(nn.Module):

    def __init__(
        self,
        in_channels: int,
        channels_7x7: int,
        conv_block: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(InceptionC, self).__init__()
        if conv_block is None:
            conv_block = BasicConv1d
        self.branch1x1 = conv_block(in_channels, 128, kernel_size=1)

        c7 = channels_7x7
        self.branch7x7_1 = conv_block(in_channels, c7, kernel_size=1)
        self.branch7x7_2 = conv_block(c7, c7, kernel_size=1, padding=0)
        self.branch7x7_3 = conv_block(c7, 128, kernel_size=7, padding=3)

        self.branch7x7dbl_1 = conv_block(in_channels, c7, kernel_size=1)
        self.branch7x7dbl_2 = conv_block(c7, c7, kernel_size=7, padding=3,)
        self.branch7x7dbl_3 = conv_block(c7, c7, kernel_size=1, padding=0,)
        self.branch7x7dbl_4 = conv_block(c7, c7, kernel_size=7, padding=3,)
        self.branch7x7dbl_5 = conv_block(c7, 128, kernel_size=1, padding=0,)

        self.branch_pool = conv_block(in_channels, 128, kernel_size=1)

    def _forward(self, x: Tensor) -> List[Tensor]:
        branch1x1 = self.branch1x1(x)

        branch7x7 = self.branch7x7_1(x)
        branch7x7 = self.branch7x7_2(branch7x7)
        branch7x7 = self.branch7x7_3(branch7x7)

        branch7x7dbl = self.branch7x7dbl_1(x)
        branch7x7dbl = self.branch7x7dbl_2(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_3(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_4(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_5(branch7x7dbl)

        branch_pool = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch7x7, branch7x7dbl, branch_pool]
        return outputs

    def forward(self, x: Tensor) -> Tensor:
        outputs = self._forward(x)
        return torch.cat(outputs, 1)
    

class SE_Block(nn.Module):
    "credits: https://github.com/moskomule/senet.pytorch/blob/master/senet/se_module.py#L4"
    def __init__(self, c, r=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool1d(1)
        self.excitation = nn.Sequential(
            nn.Linear(c, c // r, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(c // r, c, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        bs, c, _ = x.shape
        y = self.squeeze(x).view(bs, c)
        y = self.excitation(y).view(bs, c, 1)
        return x * y.expand_as(x)

In [3]:
class CFG:
    lr = 0.005
    min_lr = 1e-6
    weight_decay = 0.001
    epochs = 1200
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    seed = 42
    NFOLD = 5
    batch_size = 128
    DATA_ROOT = r'../input/2nd-solution-reproduce-feature-part/test'
    OUTPUT_DIR = r'./'
    scheduler='CosineAnnealingWarmRestarts'#
    target_col = 'time_to_eruption_normalize'
    model_name = '2nd_model'
    # for warm start
    T_0 = 10
    # for normal cosine
    T_max = 10
    num_workers = 1
    
    EARLY_STOP = True
    early_stop = 20
    
    gradient_accumulation_steps=1
    max_grad_norm = 1000
    print_freq = 20

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG.seed)

In [5]:
class MyModel(nn.Module):
    def __init__(self, in_channels=340, out_channels=128, pool_features=64):
        super(MyModel, self).__init__()
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.stem1 = nn.Sequential(nn.Conv1d(in_channels, out_channels, 3, padding=1), 
                                   nn.BatchNorm1d(out_channels), 
                                   # N,C,L => N,C,Lout
                                   nn.MaxPool1d(2,2))
        self.stem2 = nn.Sequential(nn.Conv1d(out_channels, out_channels, 3, padding=1), 
                                   nn.BatchNorm1d(out_channels), 
                                   nn.MaxPool1d(2,2))
        self.inceptiona = InceptionA(in_channels=out_channels, pool_features=pool_features)
        self.inceptionc = InceptionC(in_channels=out_channels, channels_7x7=out_channels)
        
        self.SE_Block = SE_Block(928)
        # (N, C, L)
        self.Bi_LSTM = nn.LSTM(input_size=928, 
                               hidden_size=128,
                               batch_first=True, 
                               bidirectional=True)
        self.adaptive_pooling = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(29, 1)
        
    def forward(self, x):
        x = self.bn1(x)
        x = F.leaky_relu(x)
        x = self.stem1(x)
        x = F.leaky_relu(x)
        x = self.stem2(x)
        x = F.leaky_relu(x)
        x1 = self.inceptiona(x)
        x2 = self.inceptionc(x)
        x = torch.cat([x, x1, x2], dim=1)
        x = F.leaky_relu(x)
        x = self.SE_Block(x)
        x = F.leaky_relu(x)
        x = self.Bi_LSTM(x.permute(0,2,1))
        x = self.adaptive_pooling(x[0])
        x = F.relu(x)
        x = self.fc(x.squeeze(-1))
        
        return x

In [6]:
class INGVDataset(Dataset):
    def __init__(self, df, data_root, transforms=None, output_label=True):
        super().__init__()
        self.df = df
        self.data_root = data_root
        self.transforms = transforms
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            #target = self.df.iloc[index]['label']
            target = [self.df.loc[index, 'time_to_eruption_normalize'].astype('float32')]
          
        path = os.path.join(self.data_root, str(self.df.loc[index, 'segment_id'])+'.npz')
        
        # (10, 256, 256)
        img = np.load(path)['arr_0'].astype('float32')
            
        # do label smoothing
        if self.output_label == True:
            return torch.tensor(img), torch.tensor(target)
        else:
            return torch.tensor(img)

In [7]:
df = pd.read_csv(r'../input/predict-volcanic-eruptions-ingv-oe/train.csv')
time2eruption_mean = df['time_to_eruption'].mean(axis=0)
test = pd.read_csv(r'../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')

In [8]:
check_point = torch.load(r'../input/2nd-solution-reproduce-train-part/2nd_model_best.pth')
model = MyModel()
model.load_state_dict(check_point['model'])
model.to(CFG.device)

MyModel(
  (bn1): BatchNorm1d(340, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (stem1): Sequential(
    (0): Conv1d(340, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (stem2): Sequential(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (inceptiona): InceptionA(
    (branch1x1): BasicConv1d(
      (conv): Conv1d(128, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (branch5x5_1): BasicConv1d(
      (conv): Conv1d(128, 48, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d

In [9]:
def inference(model, test):
    test_dataset = INGVDataset(test, CFG.DATA_ROOT, output_label=False)
    test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, num_workers=CFG.num_workers, shuffle=False)
    model.eval()
    preds = []
    for image in tqdm(test_loader):
        image = image.to(CFG.device)
        preds.append(model(image).detach().to('cpu').numpy()*time2eruption_mean)
    preds = np.concatenate(preds)
    
    return preds

In [10]:
preds = inference(model, test)

100%|██████████| 36/36 [00:49<00:00,  1.38s/it]


In [11]:
test['time_to_eruption'] = preds

In [12]:
test.to_csv('submission.csv', index=None)